In [ ]:
def get_files_of_type(parent_path:str, filetype:str, as_dict:bool=False) :

    assert os.path.isdir(parent_path), f'{parent_path} is not a valid directory.'

    path_list = sorted([os.path.join(root,file)
                        for root,_,files in os.walk(parent_path)
                        for file in files if file.endswith(filetype)])

    assert len(path_list) > 0, f'{parent_path} contains 0 files of file type {filetype}.'
    
    if as_dict :
        
        path_dict = {".".join(os.path.split(path)[-1].split(".")[:-1]):path for path in path_list}

        return path_dict

    return path_list
    
# Helper function convert any column with header with substring "time" to pandas.datetime
# Taken from Young Sang Choi
def dataframe_datetime(df) :
    
    for c in [c for c in df.columns if 'time' in c] :

        try :
            df[c] = pd.to_datetime(df[c])
        except Exception :
            pass

    return df

In [ ]:
import pandas as pd
import gc

# Import icu stays records 
icustays_df = pd.read_csv('/kaggle/input/data-icu/icustays.csv',low_memory=True)
# Import hospital admissions records
admissions_df = pd.read_csv('/kaggle/input/data-icu/admissions.csv',low_memory=True)
# Use atetime conversion
icustays_df = dataframe_datetime(icustays_df)
admissions_df = dataframe_datetime(admissions_df)

# Check that all hadm_ids for icu stays table are in the admissions table
assert set(icustays_df['hadm_id']).issubset(set(admissions_df['hadm_id']))

In [ ]:
icustays_df.head()

In [ ]:
admissions_df.head()

In [ ]:
import numpy as np 
# Keep icustays with lengths of stay greater or equal to 1 day (taken from admissions_df)
icustays_df = icustays_df[icustays_df['los'] >= 1]
icu_hadm_ids = set(icustays_df['hadm_id']) & set(admissions_df['hadm_id'])
admissions_df = admissions_df[admissions_df['hadm_id'].isin(icu_hadm_ids)]
subjects = set(admissions_df['subject_id'])
stays = set(icustays_df['stay_id'])
#print(f'{icustays_df.shape=}')

In [ ]:
# Obtain patient age (based on HIPAA regulations) and gender - gender, 1=female, 0=male
# Adapted from Young Sang Choi
patients_df = pd.read_csv('/kaggle/input/data-icu/patients.csv')
patients_df = patients_df[patients_df['subject_id'].isin(subjects)]
patients_df['gender'] = np.array(patients_df['gender'] == 'F').astype(int)
anchor_age_tuples = patients_df.apply(lambda row : (row['subject_id'], row['anchor_age'], row['anchor_year']), 1)
anchor_age_dict = {subject_id:{'anchor_age':anchor_age, 'anchor_year':anchor_year} for subject_id,anchor_age,anchor_year in anchor_age_tuples}
gender_dict = dict(zip(patients_df['subject_id'], patients_df['gender']))
icustays_df['age'] = icustays_df.apply(lambda row : anchor_age_dict[row['subject_id']]['anchor_age'] + (row['intime'].year - anchor_age_dict[row['subject_id']]['anchor_year']), 1)
icustays_df['age'] = icustays_df.apply(lambda row : row['age'] if row['age'] <= 90 else 90, 1) # HIPAA deidentification rule
icustays_df['gender'] = icustays_df.apply(lambda row: gender_dict[row['subject_id']], 1)

In [ ]:
# Add insurance and admission location to each subject_id in icustays_df
# Adapted from Young Sang Choi
admissions_df = admissions_df[admissions_df['subject_id'].isin(subjects)]
insurance_dict = dict(zip(admissions_df['subject_id'], admissions_df['insurance']))
icustays_df['insurance'] = icustays_df['subject_id'].map(insurance_dict)
admloc_dict = dict(zip(admissions_df['subject_id'], admissions_df['admission_location']))
icustays_df['admission location'] = icustays_df['subject_id'].map(admloc_dict)
race_dict = dict(zip(admissions_df['subject_id'], admissions_df['race']))
icustays_df['race'] = icustays_df['subject_id'].map(race_dict)


In [ ]:
# Delete tuples not needed anymore
del anchor_age_tuples
del anchor_age_dict
del gender_dict

In [ ]:
import os
# Import vitals table (chartevents)
# Adapted from Young Sang Choi
chartevent_definitions = pd.read_csv('/kaggle/input/data-icu/d_items.csv',low_memory=True)
chartevent_definitions = chartevent_definitions[(chartevent_definitions['linksto'] == 'chartevents') & (chartevent_definitions['category'] == 'Routine Vital Signs')]
routine_vital_items = chartevent_definitions['itemid'].values
print(f'Number of unique routine vital sign item ids: {len(routine_vital_items)}')

chartevents = pd.read_csv('/kaggle/input/data-icu/chartevents.csv', chunksize = 10000000)
icu_stay_ids = set(icustays_df['stay_id'])
subjects = set(icustays_df['subject_id'])


for p in [os.path.join('.', 'data'), os.path.join('.', 'data', 'mimic_chartevents')] :
    if not os.path.isdir(p) :
        os.mkdir(p)

mimic_chartevents_parent = os.path.join('.', 'data', 'mimic_chartevents')

if len(os.listdir(mimic_chartevents_parent)) == 0 : 

    for i,chunk in enumerate(chartevents) : 
        
        original_size = chunk.shape
        chunk = chunk[(chunk['stay_id'].isin(icu_stay_ids)) & (chunk['itemid'].isin(routine_vital_items))]
        chunk.to_csv(os.path.join(mimic_chartevents_parent, f'mimic_vitals_{i}.csv'), index = False)
        print(f'Chunk {i}: selected {chunk.shape[0]} from {original_size[0]} rows.')

chartevents_df = pd.concat([pd.read_csv(path) for path in get_files_of_type(mimic_chartevents_parent, filetype = 'csv')], axis=0, ignore_index=True)
chartevents_df.reset_index(drop = True, inplace = True)
chartevents_df['charttime'] = pd.to_datetime(chartevents_df['charttime'])

#print(f'{chartevents_df.shape=}')

# Group the chartevents_df by 'stay_id' and 'itemid', and keep the first row for each group - first vital signs for each patient
filtered_chartevents_df = chartevents_df.groupby(['stay_id', 'itemid']).first().reset_index()
#print(f'{filtered_chartevents_df.shape=}')

In [ ]:
# Merge the chartevents and icustay DataFrames based on the condition
filtered_chartevents_df = pd.merge(
    filtered_chartevents_df,
    icustays_df,
    how='inner',
    on=['stay_id','subject_id','hadm_id']
)
#filtered_chartevents_df.rename(columns={'stay_id_x': 'stay_id'}, inplace=True)

In [ ]:
import pandas as pd
# Obtain labevents table for all patients in cohort
labs_df = pd.read_csv('/kaggle/input/data-icu/labevents.csv',low_memory=True)
labs_df = labs_df[labs_df['subject_id'].isin(subjects)]
labs_df = dataframe_datetime(labs_df)
print(f'{labs_df.shape=}')

In [ ]:
# Merge the DataFrames based on the condition
filtered_labs_df = pd.merge(
    labs_df,
    icustays_df,
    how='inner',
    left_on=['subject_id', 'hadm_id'],
    right_on=['subject_id', 'hadm_id']
)

filtered_labs_df['charttime'] = pd.to_datetime(filtered_labs_df['charttime'])

# Filter rows where 'charttime' is between 'intime' and 'outtime'
condition = (filtered_labs_df['charttime'] >= filtered_labs_df['intime']) & (filtered_labs_df['charttime'] <= filtered_labs_df['outtime'])
filtered_labs_df.loc[condition, 'stay_id'] = filtered_labs_df.loc[condition, 'stay_id']

# Display the resulting DataFrame
#print(filtered_labs_df[['subject_id','hadm_id','stay_id','itemid','valuenum']])
#print(f'{filtered_labs_df.shape=}')

In [ ]:
# Clean and convert the 'value' column to numeric
filtered_labs_df['value'] = pd.to_numeric(filtered_labs_df['value'], errors='coerce')

# Check for duplicate itemid within each stay_id group
duplicates_for_stay_id = filtered_labs_df[filtered_labs_df.duplicated(subset=['stay_id', 'itemid'], keep=False)]

# Calculate the average value for duplicate itemid within each stay_id group
averages_for_duplicates = duplicates_for_stay_id.groupby(['stay_id', 'itemid'])['value'].mean().reset_index()

# Merge the calculated averages back into the original DataFrame
filtered_labs_df = pd.merge(filtered_labs_df, averages_for_duplicates, how='left', on=['stay_id', 'itemid'], suffixes=('', '_avg'))

# Fill NaN values in the original 'value' column with the calculated averages
filtered_labs_df['value'].fillna(filtered_labs_df['value_avg'], inplace=True)

# Drop the redundant 'value_avg' column
filtered_labs_df.drop(columns=['value_avg'], inplace=True)

In [ ]:

import pandas as pd
pharm_df = pd.read_csv('/kaggle/input/data-icu/prescriptions.csv',low_memory=True)
# Merge the DataFrames based on the condition
filtered_pharm_df = pd.merge(
    pharm_df,
    icustays_df,
    how='inner',
    left_on=['subject_id', 'hadm_id'],
    right_on=['subject_id', 'hadm_id']
)

filtered_pharm_df['starttime'] = pd.to_datetime(filtered_pharm_df['starttime'])

# Filter rows where 'charttime' is between 'intime' and 'outtime'
condition = (filtered_pharm_df['starttime'] >= filtered_pharm_df['intime']) & (filtered_pharm_df['starttime'] <= filtered_pharm_df['outtime'])
filtered_pharm_df.loc[condition, 'stay_id'] = filtered_pharm_df.loc[condition, 'stay_id']

# Display the resulting DataFrame
#print(filtered_pharm_df[['subject_id','hadm_id','stay_id','drug']])
#print(f'{filtered_pharm_df.shape=}')

In [ ]:
# Create dataframes for analysis
analysis_chartevents_df = filtered_chartevents_df[['stay_id', 'subject_id', 'itemid', 'valuenum','insurance','gender','age','race']]
analysis_labs_df = filtered_labs_df[['subject_id', 'stay_id', 'itemid', 'value','insurance','gender','age','race']]
analysis_pharm_df = filtered_pharm_df[['subject_id', 'stay_id', 'drug','insurance','gender','age','race']]

In [ ]:
# Only keep columns needed for each dataframe 
_chartevents_df = filtered_chartevents_df[['stay_id', 'subject_id', 'itemid', 'valuenum']]
_labs_df = filtered_labs_df[['subject_id', 'stay_id', 'itemid', 'value']]
_pharm_df = filtered_pharm_df[['subject_id', 'stay_id', 'drug']]

In [ ]:
# Replace item_id values with names to be more legible 

d_items = pd.read_csv('/kaggle/input/data-icu/d_items.csv',low_memory=True)
# Create a dictionary mapping itemid to label from chartevent_definitions
itemid_to_label = d_items.set_index('itemid')['label'].to_dict()
# In-place replace the itemid column with label
_chartevents_df['itemid'] = _chartevents_df['itemid'].map(itemid_to_label)

# Create a dictionary mapping itemid to label from d_labitems
d_labitems = pd.read_csv('/kaggle/input/data-icu/d_labitems.csv',low_memory=True)
itemid_to_label = d_labitems.set_index('itemid')['label'].to_dict()
# In-place replace the itemid column with label
_labs_df['itemid'] = _labs_df['itemid'].map(itemid_to_label)

In [ ]:
# Pivot the DataFrame without itemid and fill NaN with 0
final_chartevents = _chartevents_df.pivot_table(
    index=['subject_id', 'stay_id'],
    columns='itemid',
    values='valuenum',
    fill_value=0  # Replace NaN with 0
)

# Reset the index if needed
final_chartevents = final_chartevents.reset_index()

# Display the resulting DataFrame
#final_chartevents.head(1)

In [ ]:
# Convert 'value' column to numeric, coerce non-numeric values to NaN
_labs_df['value'] = pd.to_numeric(_labs_df['value'], errors='coerce')

# Pivot the DataFrame without itemid and fill NaN with 0
final_labs = _labs_df.pivot_table(
    index=['subject_id', 'stay_id'],
    columns='itemid',
    values='value',
    fill_value=0  # Replace NaN with 0
)

# Reset the index if needed
final_labs = final_labs.reset_index()

# Display the resulting DataFrame
#final_labs.head(1)

In [ ]:
# Create a new column 'value' and set it to 1 for all rows
_pharm_df['value'] = 1

# Pivot the DataFrame
final_pharm = _pharm_df.pivot_table(index=['subject_id', 'stay_id'], columns='drug', values='value', fill_value=0)

# Reset the index if needed
final_pharm = final_pharm.reset_index()
#final_pharm.head(5)

In [ ]:
# Merge the first two pivoted DataFrames
icu_data = pd.merge(icustays_df, final_chartevents, on=['subject_id', 'stay_id'], how='inner')

# Merge the third pivoted DataFrame
icu_data = pd.merge(icu_data, final_labs, on=['subject_id', 'stay_id'], how='inner')

# Merge the icustays_df DataFrame
icu_data = pd.merge(icu_data, final_pharm, on=['subject_id', 'stay_id'], how='inner')

# Display the resulting DataFrame
icu_data.head(5)

In [ ]:
print(icu_data['stay_id'].nunique())

In [ ]:
# Mapping dictionary for race renaming
race_mapping = {
    'ASIAN - ASIAN INDIAN': 'ASIAN',
    'ASIAN - CHINESE': 'ASIAN',
    'ASIAN - KOREAN': 'ASIAN',
    'ASIAN - SOUTH EAST ASIAN': 'ASIAN',
    'UNKNOWN': 'OTHER',
    'UNABLE TO OBTAIN': 'OTHER',
    'PATIENT DECLINED TO ANSWER': 'OTHER',
    'WHITE - RUSSIAN': 'WHITE',
    'PORTUGUESE': 'WHITE',
    'WHITE - BRAZILIAN': 'WHITE',
    'WHITE - OTHER EUROPEAN': 'WHITE',
    'WHITE - EASTERN EUROPEAN': 'WHITE', 
    'BLACK/CAPE VERDEAN':'BLACK/AFRICAN AMERICAN',
    'BLACK/AFRICAN':'BLACK/AFRICAN AMERICAN',
    'BLACK/CARIBBEAN ISLAND':'BLACK/AFRICAN AMERICAN',
    'HISPANIC/LATINO - CUBAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - PUERTO RICAN': 'HISPANIC/LATINO',
    'HISPANIC OR LATINO': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - SALVADORAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - DOMINICAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - HONDURAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - COLUMBIAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - GUATEMALAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - CENTRAL AMERICAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - MEXICAN': 'HISPANIC/LATINO',
    'SOUTH AMERICAN': 'HISPANIC/LATINO',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'PACIFIC ISLANDER',
    'AMERICAN INDIAN/ALASKA NATIVE': 'PACIFIC ISLANDER',
    'NATIVE HAWAIIAN/PACIFIC ISLANDER': 'PACIFIC ISLANDER'
    
}
# Replace values in the 'race' column using the mapping dictionary for all dataframes 
icu_data['race'] = icu_data['race'].replace(race_mapping)
analysis_chartevents_df['race'] = analysis_chartevents_df['race'].replace(race_mapping)
analysis_labs_df['race'] = analysis_labs_df['race'].replace(race_mapping)
analysis_pharm_df['race'] = analysis_pharm_df['race'].replace(race_mapping)


In [ ]:
# Shape of the cleaned data
print(icu_data.shape[0])
print(icu_data.shape[1])

In [ ]:
# Export csv file with all the cleaned data
icu_data.to_csv('icu_data_Nolm.csv', index=False)

# Dataset Stadistics

In [ ]:
import pandas as pd

icu_data = pd.read_csv('/kaggle/working/icu_data_Nolm.csv',low_memory=True)
icu_stays = pd.read_csv('/kaggle/input/data-icu/icustays.csv', low_memory=True)
admissions = pd.read_csv('/kaggle/input/data-icu/admissions.csv',low_memory=True)
patients = pd.read_csv('/kaggle/input/data-icu/patients.csv', low_memory=True)

In [ ]:
def percentage(df, column, condition):

    condition_number = len(df.loc[df[column]==condition])
    result = condition_number / len(df)
    
    return result

In [ ]:
icu_stays.columns

In [ ]:
admissions['insurance'].unique()

In [ ]:
percentage(admissions, 'insurance', 'Other')

In [ ]:

icu_data['hadm_id'].nunique()

In [ ]:
icu_stays['hadm_id'].loc[icu_stays['los']>1].nunique()

In [ ]:
len(icu_data[icu_data['age'] >= 18])

In [ ]:
analysis_labs_df.head(1)

# Data Visualizations for Disparities in Data

## Fig. 1: Depiction of Race breakdown

### (a) Average number of medications per ICU stay

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you have a DataFrame called analysis_chartevents_df

# Group by 'stay_id' and 'insurance_grouping' and calculate the average number of chart events
average_pharm = analysis_pharm_df.groupby(['stay_id', 'race']).size().reset_index(name='pharm_count').groupby('race')['pharm_count'].mean()

# Plotting the bar chart
ax = average_pharm.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10,5))

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Medications per first 24hr ICU Stay by Race')
plt.xlabel('Race')
plt.ylabel('Average Number of Medications Taken')
plt.show()

### (b) average number of labs per ICU stay

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you have a DataFrame called analysis_chartevents_df

# Group by 'stay_id' and 'insurance_grouping' and calculate the average number of chart events
average_labs = analysis_labs_df.groupby(['stay_id', 'race']).size().reset_index(name='labs_count').groupby('race')['labs_count'].mean()

# Plotting the bar chart
ax = average_labs.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10,5))

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Labs per first 24hr ICU Stay by Race')
plt.xlabel('Race')
plt.ylabel('Average Number of Labs Taken')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
plt.show()

### (c) average number of vitals per ICU stay

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Group by 'stay_id' and 'insurance_grouping' and calculate the average number of chart events
average_chartevents = analysis_chartevents_df.groupby(['stay_id', 'race']).size().reset_index(name='chartevents_count').groupby('race')['chartevents_count'].mean()

# Plotting the bar chart
ax = average_chartevents.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10,5))

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Vitals Taken per first 24hr ICU Stay by Race')
plt.xlabel('Race')
plt.ylabel('Average Number of Vitals Taken')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
plt.show()

## Fig. 2: Depiction of gender breakdown

### (a) The average number of medications per ICU stay

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you have a DataFrame called analysis_chartevents_df

# Group by 'stay_id' and 'insurance_grouping' and calculate the average number of chart events
average_pharm = analysis_pharm_df.groupby(['stay_id', 'gender']).size().reset_index(name='pharm_count').groupby('gender')['pharm_count'].mean()

# Plotting the bar chart
ax = average_pharm.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10,5))

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Medications Taken per first 24hr ICU Stay by Gender')
plt.xlabel('Gender')
plt.ylabel('Average Number of Medications Taken')
ax.set_xticklabels(['Male', 'Female'], rotation=90, ha='right')
plt.show()

### b) the average number of labs per ICU stay

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Group by 'stay_id' and 'insurance_grouping' and calculate the average number of labs
average_labs = analysis_labs_df.groupby(['stay_id', 'gender']).size().reset_index(name='labs_count').groupby('gender')['labs_count'].mean()

# Plotting the bar chart
ax = average_labs.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10, 5))

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Labs per first 24hr ICU Stay by Gender')
plt.xlabel('Gender')
plt.ylabel('Average Number of Labs Taken')
# Combine rotation and custom labels
ax.set_xticklabels(['Male', 'Female'], rotation=90, ha='right')

plt.show()

### (c) the average number of vitals per ICU stay

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Group by 'stay_id' and 'insurance_grouping' and calculate the average number of chart events
average_chartevents = analysis_chartevents_df.groupby(['stay_id', 'gender']).size().reset_index(name='chartevents_count').groupby('gender')['chartevents_count'].mean()

# Plotting the bar chart
ax = average_chartevents.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10,5))

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Vitals Taken per first 24hr ICU Stay by Gender')
plt.xlabel('Gender Grouping')
plt.ylabel('Average Number of Vitals Taken')
ax.set_xticklabels(['Male', 'Female'], rotation=90, ha='right')
plt.show()

## Fig. 3: Depiction of insurance breakdown

### (a) Average number of medications per ICU stay

In [ ]:
# Assuming you have a DataFrame called analysis_chartevents_df

# Group by 'stay_id' and 'insurance_grouping' and calculate the average number of chart events
average_pharm = analysis_pharm_df.groupby(['stay_id', 'gender']).size().reset_index(name='pharm_count').groupby('gender')['pharm_count'].mean()

# Plotting the bar chart
ax = average_pharm.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10,5))

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Medications Taken per first 24hr ICU Stay by Gender')
plt.xlabel('Gender')
plt.ylabel('Average Number of Medications Taken')
ax.set_xticklabels(['Male', 'Female'], rotation=90, ha='right')
plt.show()

### (b) average number of labs per ICU stay

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Group by 'stay_id' and 'insurance_grouping' and calculate the average number of chart events
average_labs = analysis_labs_df.groupby(['stay_id', 'insurance']).size().reset_index(name='labs_count').groupby('insurance')['labs_count'].mean()

# Plotting the bar chart
ax = average_labs.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10, 5))

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Labs per first 24hr ICU Stay by Insurance')
plt.xlabel('Insurance')
plt.ylabel('Average Number of Labs Taken')
# Rotate x-axis labels
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
plt.show()


### (c) average number of vitals per ICU stay

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Group by 'stay_id' and 'insurance_grouping' and calculate the average number of chart events
average_chartevents = analysis_chartevents_df.groupby(['stay_id', 'insurance']).size().reset_index(name='chartevents_count').groupby('insurance')['chartevents_count'].mean()

# Plotting the bar chart
ax = average_chartevents.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10, 5))

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Vitals Taken per first 24hr ICU Stay by Insurance')
plt.xlabel('Insurance')
plt.ylabel('Average Number of Vitals Taken')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
plt.show()

## Fig. 4: Length of ICU Stay by Insurance and Race

In [ ]:
import seaborn as sns
# Assuming you have a DataFrame called icu_data

# Set seaborn style to whitegrid
sns.set_style("whitegrid")

# Plotting the grouped bar plot for average LOS
plt.figure(figsize=(20, 8))
ax = sns.barplot(x='race', y='los', hue='insurance', data=icu_data, palette='colorblind', estimator=np.mean)

plt.title('Average Length of Stay (LOS) by Insurance and Race')
plt.xlabel('Race')
plt.ylabel('Average Length of Stay (LOS)')

plt.legend(title='Insurance', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Assuming you have a DataFrame called icu_data

# Set seaborn style to whitegrid
sns.set_style("whitegrid")

# Plotting the grouped bar plot for average LOS
plt.figure(figsize=(20, 8))
ax = sns.barplot(x='insurance', y='los', hue='race', data=icu_data, palette='colorblind', estimator=np.mean)

plt.title('Average Length of Stay (LOS) by Insurance and Race')
plt.xlabel('Race')
plt.ylabel('Average Length of Stay (LOS)')

plt.legend(title='Race', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# ANOVA ANALYSIS

In [ ]:
grouped_data = icu_data.groupby(['gender', 'insurance', 'race'])['los'].mean()

# To calculate median instead of mean, use the following:
# grouped_data = icustays_df.groupby(['gender', 'insurance', 'race'])['los'].median()

# Displaying the grouped data
print(grouped_data)

In [ ]:
from scipy.stats import f_oneway

# Assuming 'icustays_df' is your DataFrame
# Performing one-way ANOVA test for 'gender'
anova_gender = f_oneway(*[group['los'].values for name, group in icu_data.groupby('gender')])

# Performing one-way ANOVA test for 'insurance'
anova_insurance = f_oneway(*[group['los'].values for name, group in icu_data.groupby('insurance')])

# Performing one-way ANOVA test for 'race'
anova_race = f_oneway(*[group['los'].values for name, group in icu_data.groupby('race')])

# Displaying the ANOVA results
print("ANOVA for Gender:")
print(anova_gender)

print("\nANOVA for Insurance:")
print(anova_insurance)

print("\nANOVA for Race:")
print(anova_race)

# Model XGBOOST - Replication

In [ ]:
condition = (icu_data['los'] > 4).map({True: 'Long', False: 'Short'})

# Create a new column 'los_type' based on the condition
icu_data['los_type'] = condition

In [ ]:
# Split data into training and validation data
from sklearn.model_selection import train_test_split

# Assuming 'subject_id' is the patient identifier
patient_ids = icu_data['subject_id'].unique()

# Split patient_ids into training and testing sets
train_patient_ids, validation_patient_ids = train_test_split(patient_ids, test_size=0.2, random_state=42)

# Filter the dataframe based on the split of patient_ids
train_data = icu_data[icu_data['subject_id'].isin(train_patient_ids)]
validation_data = icu_data[icu_data['subject_id'].isin(validation_patient_ids)]

# Check the shapes of the training and testing sets
print("Training set shape:", train_data.shape)
print("Validation set shape:", validation_data.shape)

In [ ]:
train_insurance_counts =train_data['insurance'].value_counts()
print("Train Data Insurance Counts:")
print(train_insurance_counts)

validation_insurance_counts =validation_data['insurance'].value_counts()
print("Validation Data Insurance Counts:")
print(validation_insurance_counts)

train_race_counts =train_data['race'].value_counts()
print("Train Data Race Counts:")
print(train_race_counts)

validation_race_counts =validation_data['race'].value_counts()
print("Validation Data Race Counts:")
print(validation_race_counts)

train_duration_counts =train_data['los_type'].value_counts()
print("Train Data LOS Counts:")
print(train_duration_counts)

validation_duration_counts =validation_data['los_type'].value_counts()
print("Validation Data LOS Counts:")
print(validation_duration_counts)

In [ ]:
# Assuming 'subject_id' is the patient identifier
patient_ids_train = train_data['subject_id'].unique()

# Split patient_ids_train into training and testing sets
train_train_ids, test_train_ids = train_test_split(patient_ids_train, test_size=0.2, random_state=42)

# Filter the train_data based on the split of patient_ids_train
train_train_data = train_data[train_data['subject_id'].isin(train_train_ids)]
test_train_data = train_data[train_data['subject_id'].isin(test_train_ids)]

# Check the shapes of the training and testing sets
print("Train Training set shape:", train_train_data.shape)
print("Train Testing set shape:", test_train_data.shape)

In [ ]:
!pip install xgboost
import pandas as pd
import xgboost
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Columns to exclude from features
exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'insurance', 'race', 'PHYSICIAN REFERRAL']

# Define your feature columns (exclude certain columns)
feature_columns = [col for col in train_data.columns if col not in exclude_columns]

# Exclude non-numeric columns before scaling
numeric_columns = train_train_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X_train = train_train_data[numeric_columns]
y_train = train_train_data['los_type']
X_test = test_train_data[numeric_columns]
y_test = test_train_data['los_type']

# Convert 'Short' and 'Long' to numerical labels (0 and 1)
y_train = y_train.map({'Short': 0, 'Long': 1})
y_test = y_test.map({'Short': 0, 'Long': 1})

# Standardize the features if needed (optional but often beneficial)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an XGBoost classifier
model = XGBClassifier(objective='binary:logistic', random_state=42)

# Train the model
model.fit(X_train, y_train)
# Make predictions on the test set
y_pred_prob = model.predict_proba(X_test)[:, 1]

# Convert probability scores to binary predictions using a threshold
threshold = 0.5
y_pred = (y_pred_prob >= threshold).astype(int)

# Generate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Generate precision-recall curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
average_precision = average_precision_score(y_test, y_pred_prob)


In [ ]:
# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Plot precision-recall curve
plt.figure(figsize=(8, 6))
plt.step(recall, precision, color='b', where='post', label='Precision-Recall curve (AP = {:.2f})'.format(average_precision))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='upper right')
plt.show()

print(f'Area under ROC curve (AUC-ROC): {roc_auc:.2f}')
print(f'Area under Precision-Recall curve (AUC-PR): {average_precision:.2f}')

In [ ]:
# Generate confusion matrix and classification report
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

print("Confusion Matrix:")
print(cm)

print("\nClassification Report:")
print(cr)

In [ ]:
# Columns to exclude from features
exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'insurance', 'race', 'PHYSICIAN REFERRAL']

# Define your feature columns (exclude certain columns)
feature_columns = [col for col in train_data.columns if col not in exclude_columns]

# Exclude non-numeric columns before scaling
numeric_columns = train_train_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X_train = train_train_data[numeric_columns]
y_train = train_train_data['los_type']
X_test = test_train_data[numeric_columns]
y_test = test_train_data['los_type']

# Convert 'Short' and 'Long' to numerical labels (0 and 1)
y_train = y_train.map({'Short': 0, 'Long': 1})
y_test = y_test.map({'Short': 0, 'Long': 1})

# Standardize the features if needed (optional but often beneficial)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Extract insurance information for the test set
insurance_test = train_data.loc[y_test.index, 'insurance']

# Create an XGBoost classifier
model = XGBClassifier(objective='binary:logistic', random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Combine predictions and insurance information into a DataFrame
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Insurance': insurance_test})

# Assess performance for each insurance type
insurance_types = results_df['Insurance'].unique()
for insurance_type in insurance_types:
    subset = results_df[results_df['Insurance'] == insurance_type]
    
    # Generate confusion matrix and classification report
    cm = confusion_matrix(subset['Actual'], subset['Predicted'])
    cr = classification_report(subset['Actual'], subset['Predicted'])
    
    print(f"Results for {insurance_type} insurance:")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(cr)
    print('\n' + '-'*50 + '\n')

In [ ]:
# Columns to exclude from features
exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'insurance', 'race', 'PHYSICIAN REFERRAL']

# Define your feature columns (exclude certain columns)
feature_columns = [col for col in train_data.columns if col not in exclude_columns]

# Exclude non-numeric columns before scaling
numeric_columns = train_train_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X_train = train_train_data[numeric_columns]
y_train = train_train_data['los_type']
X_test = test_train_data[numeric_columns]
y_test = test_train_data['los_type']

# Convert 'Short' and 'Long' to numerical labels (0 and 1)
y_train = y_train.map({'Short': 0, 'Long': 1})
y_test = y_test.map({'Short': 0, 'Long': 1})

# Standardize the features if needed (optional but often beneficial)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Extract insurance information for the test set
race_test = train_data.loc[y_test.index, 'race']

# Create an XGBoost classifier
model = XGBClassifier(objective='binary:logistic', random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Combine predictions and insurance information into a DataFrame
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Race': race_test})

# Get unique race types
race_types = results_df['Race'].unique()
# Assess performance for each race type
for race_type in race_types:
    subset = results_df[results_df['Race'] == race_type]
    
    # Generate confusion matrix and classification report
    cm = confusion_matrix(subset['Actual'], subset['Predicted'])
    cr = classification_report(subset['Actual'], subset['Predicted'])
    
    print(f"Results for {race_type} race:")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(cr)
    print('\n' + '-'*50 + '\n')

In [ ]:
def equalized_odds(y_true, y_pred, sensitive_feature):
    # Calculate confusion matrices
    cm_protected = confusion_matrix(y_true[sensitive_feature], y_pred[sensitive_feature])
    cm_non_protected = confusion_matrix(y_true[~sensitive_feature], y_pred[~sensitive_feature])

    # Calculate True Positive Rate (sensitivity) and False Positive Rate (fallout) for both groups
    tpr_protected = cm_protected[1, 1] / cm_protected.sum(axis=1)[1]
    fpr_protected = cm_protected[0, 1] / cm_protected.sum(axis=1)[0]

    tpr_non_protected = cm_non_protected[1, 1] / cm_non_protected.sum(axis=1)[1]
    fpr_non_protected = cm_non_protected[0, 1] / cm_non_protected.sum(axis=1)[0]

    # Calculate the absolute difference in TPR and FPR between protected and non-protected groups
    tpr_difference = abs(tpr_protected - tpr_non_protected)
    fpr_difference = abs(fpr_protected - fpr_non_protected)

    return tpr_difference + fpr_difference

def demographic_parity(y_pred, sensitive_feature):
    proportion_protected = y_pred[sensitive_feature].mean()
    proportion_non_protected = y_pred[~sensitive_feature].mean()

    return abs(proportion_protected - proportion_non_protected)

In [ ]:
# Columns to exclude from features
exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'insurance', 'race', 'PHYSICIAN REFERRAL']

# Define your feature columns (exclude certain columns)
feature_columns = [col for col in validation_data.columns if col not in exclude_columns]

# Exclude non-numeric columns before scaling
numeric_columns = validation_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X_verify = validation_data[numeric_columns]
y_verify = validation_data['los_type']

# Convert 'Short' and 'Long' to numerical labels (0 and 1)
y_verify = y_verify.map({'Short': 0, 'Long': 1})

# Standardize the features using the same scaler from the training set
X_verify = scaler.transform(X_verify)

# Make predictions on the verification set using the existing model
y_verify_pred_prob = model.predict_proba(X_verify)[:, 1]

# Convert probability scores to binary predictions using a threshold
y_verify_pred = (y_verify_pred_prob > threshold).astype(int)

# Generate ROC curve for the verification set
fpr_verify, tpr_verify, thresholds_verify = roc_curve(y_verify, y_verify_pred_prob)
roc_auc_verify = auc(fpr_verify, tpr_verify)

# Generate precision-recall curve for the verification set
precision_verify, recall_verify, _ = precision_recall_curve(y_verify, y_verify_pred_prob)
average_precision_verify = average_precision_score(y_verify, y_verify_pred_prob)


In [ ]:
# Plot ROC curve for the verification set
plt.figure(figsize=(8, 6))
plt.plot(fpr_verify, tpr_verify, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc_verify))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve - Validation Set')
plt.legend(loc='lower right')
plt.show()

# Plot precision-recall curve for the verification set
plt.figure(figsize=(8, 6))
plt.step(recall_verify, precision_verify, color='b', where='post', label='Precision-Recall curve (AP = {:.2f})'.format(average_precision_verify))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Validation Set')
plt.legend(loc='upper right')
plt.show()

print(f'Area under ROC curve (AUC-ROC) for Verification Set: {roc_auc_verify:.2f}')
print(f'Area under Precision-Recall curve (AUC-PR) for Verification Set: {average_precision_verify:.2f}')

# Generate confusion matrix and classification report for the verification set
cm_verify = confusion_matrix(y_verify, y_verify_pred)
cr_verify = classification_report(y_verify, y_verify_pred)

print("Confusion Matrix for Validation Set:")
print(cm_verify)

print("\nClassification Report for Validation Set:")
print(cr_verify)

In [ ]:
# Assuming you have the true labels for the validation set
y_true = validation_data['los_type'].map({'Short': 0, 'Long': 1})

# Assuming 'validation_data' has predictions stored in 'y_verify_pred'
cm_verify = confusion_matrix(y_true, y_verify_pred)

# Extract values from the confusion matrix
tn, fp, fn, tp = cm_verify.ravel()

# Calculate equalized odds
equalized_odds_race = (tp / (tp + fn)) / (fp / (fp + tn))

print(equalized_odds_race)

In [ ]:
# Columns to exclude from features
exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'race', 'PHYSICIAN REFERRAL']

# Define your feature columns (exclude certain columns)
feature_columns = [col for col in validation_data.columns if col not in exclude_columns]

# Exclude non-numeric columns before scaling
numeric_columns = validation_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X = validation_data[numeric_columns]
y = validation_data['los_type']

# Convert 'Short' and 'Long' to numerical labels (0 and 1)
y = y.map({'Short': 0, 'Long': 1})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Extract insurance information for the test set
insurance_test = validation_data.loc[y_test.index, 'insurance']

# Standardize the features if needed (optional but often beneficial)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an XGBoost classifier
model = XGBClassifier(objective='binary:logistic', random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Combine predictions and insurance information into a DataFrame
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Insurance': insurance_test})

# Assess performance for each insurance type
insurance_types = results_df['Insurance'].unique()
for insurance_type in insurance_types:
    subset = results_df[results_df['Insurance'] == insurance_type]
    
    # Generate confusion matrix and classification report
    cm = confusion_matrix(subset['Actual'], subset['Predicted'])
    cr = classification_report(subset['Actual'], subset['Predicted'])
    
    print(f"Results for {insurance_type} insurance:")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(cr)
    print('\n' + '-'*50 + '\n')

In [ ]:
# Columns to exclude from features
exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'race', 'PHYSICIAN REFERRAL']

# Define your feature columns (exclude certain columns)
feature_columns = [col for col in validation_data.columns if col not in exclude_columns]

# Exclude non-numeric columns before scaling
numeric_columns = validation_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X = validation_data[numeric_columns]
y = validation_data['los_type']

# Convert 'Short' and 'Long' to numerical labels (0 and 1)
y = y.map({'Short': 0, 'Long': 1})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Extract insurance information for the test set
insurance_test = validation_data.loc[y_test.index, 'insurance']

# Standardize the features if needed (optional but often beneficial)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an XGBoost classifier
model = XGBClassifier(objective='binary:logistic', random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Combine predictions and insurance information into a DataFrame
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Insurance': insurance_test})

# Assess performance for each insurance type
insurance_types = results_df['Insurance'].unique()

for insurance_type in insurance_types:
    subset = results_df[results_df['Insurance'] == insurance_type]
    
    # Generate confusion matrix and classification report
    cm = confusion_matrix(subset['Actual'], subset['Predicted'])
    cr = classification_report(subset['Actual'], subset['Predicted'])
    
    print(f"Results for {insurance_type} insurance:")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(cr)

    # Calculate Type 1 and Type 2 errors
    type1_error = cm[0, 1]  # False Positives
    type2_error = cm[1, 0]  # False Negatives

    print(f"Type 1 Error (False Positives): {type1_error}")
    print(f"Type 2 Error (False Negatives): {type2_error}")

    print('\n' + '-'*50 + '\n')

In [ ]:
# Columns to exclude from features
exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'race', 'PHYSICIAN REFERRAL']

# Define your feature columns (exclude certain columns)
feature_columns = [col for col in validation_data.columns if col not in exclude_columns]

# Exclude non-numeric columns before scaling
numeric_columns = validation_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X = validation_data[numeric_columns]
y = validation_data['los_type']

# Convert 'Short' and 'Long' to numerical labels (0 and 1)
y = y.map({'Short': 0, 'Long': 1})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Extract insurance information for the test set
race_test = validation_data.loc[y_test.index, 'race']

# Standardize the features if needed (optional but often beneficial)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an XGBoost classifier
model = XGBClassifier(objective='binary:logistic', random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Combine predictions and insurance information into a DataFrame
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Race': race_test})

# Assess performance for each race type
for race_type in race_types:
    subset = results_df[results_df['Race'] == race_type]
    
    # Generate confusion matrix and classification report
    cm = confusion_matrix(subset['Actual'], subset['Predicted'])
    cr = classification_report(subset['Actual'], subset['Predicted'])
    
    print(f"Results for {race_type} race:")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(cr)
    print('\n' + '-'*50 + '\n')